In [2]:
from sklearn.ensemble import RandomForestClassifier 
RF_C = RandomForestClassifier(
    #n_estimators=20,
    max_features="auto",
    criterion='gini',
    #n_jobs=4,
    #oob_score=True
    ) 
RF_C.fit(train_data_padding, list_label_train) 
#int
predict_rfc = RF_C.predict(test_data_padding)   

In [1]:
#LOAD 
def prepare_label(dict_data)-> list:
    """
        dict_data : { e_id: {
                            'log_features':[] ,
                            'info_features':[] ,
                            'label' : 1 or 0
                            } 
                    }

    Returns:
        list_assemble_data : [[ *log_features , *info_features ],......]
        list_e_id : [ e_id_1 ,e_id_2,...... ]
        list_label : [ 0, 1, ..........]
    """    
    list_assemble_data = []
    list_e_id = []
    list_label = []
    for e_id,dict_ in dict_data.items():

        log_data  = dict_['log_features']
        info_data = dict_['info_features']
        label     = dict_['label']

        list_assemble_data.append([ *log_data,*info_data ])
        list_e_id.append(int(e_id))
        list_label.append(int(label))

    return list_assemble_data , list_e_id , list_label
def assemble_predictedal_and_predict_label(
    list_label,
    list_e_id,
    predict_label)-> dict:
    """
    Returns:
        dict: { e_id:[ predictedal_label , predict_label ]}
    """    
    dict_ori_and_predict_label = {}
    for row in range(len(list_label)):
        e_id = list_e_id[row]
        ori_label = list_label[row]
        dict_ori_and_predict_label[e_id] = [ori_label,predict_label[row]]

    return dict_ori_and_predict_label

def predict_label_to_int(predict_label_list,threshold):

    predict_label_int = []
    for i in predict_label:
        value= i
        if value >threshold:label_ = int(1)
        else:label_ = int(0)
        predict_label_int.append(label_)

    return predict_label_int
def measure(predict_label_int,list_label_test):
    f1             = f1_score(predict_label_int,list_label_test)
    accuracy = accuracy_score(predict_label_int,list_label_test)
    AUC =       roc_auc_score(predict_label_int,list_label_test)
    # precision = precision_score(predict_label_int,test_label.tolist())
    # recall = recall_score(predict_label_int,test_label.tolist())

    print(round(f1,4),round(AUC,4))

import json
dict_data_train = json.load(open('after_processed_data_file\\dict_data_train_for_analy_fix_birth.json','r'))
dict_data_test = json.load(open('after_processed_data_file\\dict_data_test_for_analy_fix_birth.json','r'))

list_data_train,list_e_id_train,list_label_train = prepare_label(dict_data_train)
list_data_test,list_e_id_test,list_label_test = prepare_label(dict_data_test)

import numpy as np
train_data =np.array( list_data_train)
test_data  =np.array( list_data_test)
from sklearn.impute import SimpleImputer
Padding = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_data_padding = Padding.fit_transform(train_data)
test_data_padding  = Padding.fit_transform(test_data)


In [3]:
from sklearn.ensemble import GradientBoostingClassifier  #梯度提升树
GDBT = GradientBoostingClassifier(
   n_estimators=10,learning_rate=0.1,random_state=10
    )
GDBT.fit(train_data_padding,list_label_train)
# int
predict_gbdt = GDBT.predict(test_data_padding)

In [4]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(
   # max_iter = 100,solver='liblinear'
    )
LR.fit(train_data_padding,list_label_train)
# int
predict_lr = LR.predict(test_data_padding)

In [5]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [7]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
for result in [
    predict_lr,
    predict_gbdt,
    predict_rfc,
   # predict_svm
    ]:
    measure(
        predict_label_int=result,
        list_label_test=list_label_test)



0.8583 0.5553
0.8925 0.8226
0.9015 0.8047


In [10]:
import sklearn.model_selection  as Kfold
accs= Kfold.cross_validate(
    LR, train_data_padding, 
    y=list_label_train, 
    scoring=['roc_auc','f1'],cv=3, 
    n_jobs=1)



In [15]:
accs

{'fit_time': array([1.85818458, 2.14755273, 1.83001685, 1.87541986, 2.3081181 ,
        2.37367916, 2.3312583 , 1.3854022 , 2.00444174, 2.21884489]),
 'score_time': array([0.05012918, 0.04400444, 0.03900266, 0.04000378, 0.05300307,
        0.04310679, 0.03201103, 0.02400374, 0.04252362, 0.04300189]),
 'test_roc_auc': array([0.54731275, 0.45574588, 0.63490249, 0.61816644, 0.43423431,
        0.5001949 , 0.59820735, 0.59753766, 0.51308725, 0.56485497]),
 'test_f1': array([0.85996227, 0.85759977, 0.85966377, 0.86003348, 0.85692464,
        0.86085128, 0.85951974, 0.85779766, 0.8592711 , 0.85677935])}

In [16]:
import sklearn.model_selection  as Kfold
accs= Kfold.cross_validate(
    LR, train_data_padding, 
    y=list_label_train, 
    scoring=['auc','f1'],cv=10, 
    n_jobs=-1)



ValueError: 'auc' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

In [17]:
sklearn.metrics.SCORERS.keys()

NameError: name 'sklearn' is not defined

In [18]:
import sklearn

In [19]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [20]:
predict_lr = LR.predict(test_data_padding)

In [21]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
for result in [
    predict_lr,
   # predict_gbdt,
   # predict_rfc,
   # predict_svm
    ]:
    measure(
        predict_label_int=result,
        list_label_test=list_label_test)


0.8583 0.5553


In [22]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(
   # max_iter = 100,solver='liblinear'
    )
LR.fit(train_data_padding,list_label_train)
# int
predict_lr = LR.predict(test_data_padding)

In [23]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
for result in [
    predict_lr,
   # predict_gbdt,
   # predict_rfc,
   # predict_svm
    ]:
    measure(
        predict_label_int=result,
        list_label_test=list_label_test)


0.8583 0.5553


In [24]:
#LOAD 
def prepare_label(dict_data)-> list:
    """
        dict_data : { e_id: {
                            'log_features':[] ,
                            'info_features':[] ,
                            'label' : 1 or 0
                            } 
                    }

    Returns:
        list_assemble_data : [[ *log_features , *info_features ],......]
        list_e_id : [ e_id_1 ,e_id_2,...... ]
        list_label : [ 0, 1, ..........]
    """    
    list_assemble_data = []
    list_e_id = []
    list_label = []
    for e_id,dict_ in dict_data.items():

        log_data  = dict_['log_features']
        info_data = dict_['info_features']
        label     = dict_['label']

        list_assemble_data.append([ *log_data,*info_data ])
        list_e_id.append(int(e_id))
        list_label.append(int(label))

    return list_assemble_data , list_e_id , list_label
def assemble_predictedal_and_predict_label(
    list_label,
    list_e_id,
    predict_label)-> dict:
    """
    Returns:
        dict: { e_id:[ predictedal_label , predict_label ]}
    """    
    dict_ori_and_predict_label = {}
    for row in range(len(list_label)):
        e_id = list_e_id[row]
        ori_label = list_label[row]
        dict_ori_and_predict_label[e_id] = [ori_label,predict_label[row]]

    return dict_ori_and_predict_label

def predict_label_to_int(predict_label_list,threshold):

    predict_label_int = []
    for i in predict_label:
        value= i
        if value >threshold:label_ = int(1)
        else:label_ = int(0)
        predict_label_int.append(label_)

    return predict_label_int
def measure(predict_label_int,list_label_test):
    f1             = f1_score(predict_label_int,list_label_test)
    accuracy = accuracy_score(predict_label_int,list_label_test)
    AUC =       roc_auc_score(predict_label_int,list_label_test)
    # precision = precision_score(predict_label_int,test_label.tolist())
    # recall = recall_score(predict_label_int,test_label.tolist())

    print(round(f1,4),round(AUC,4))

import json
dict_data_train = json.load(open('after_processed_data_file\\dict_data_train_for_analy_fix_birth.json','r'))
dict_data_test = json.load(open('after_processed_data_file\\dict_data_test_for_analy_fix_birth.json','r'))

list_data_train,list_e_id_train,list_label_train = prepare_label(dict_data_train)
list_data_test,list_e_id_test,list_label_test = prepare_label(dict_data_test)

import numpy as np
train_data =np.array( list_data_train)
test_data  =np.array( list_data_test)
from sklearn.impute import SimpleImputer
Padding = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_data_padding = Padding.fit_transform(train_data)
test_data_padding  = Padding.fit_transform(test_data)


In [25]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(
   # max_iter = 100,solver='liblinear'
    )
LR.fit(train_data_padding,list_label_train)
# int
predict_lr = LR.predict(test_data_padding)

In [26]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
for result in [
    predict_lr,
   # predict_gbdt,
   # predict_rfc,
   # predict_svm
    ]:
    measure(
        predict_label_int=result,
        list_label_test=list_label_test)


0.8583 0.5553


In [27]:
from sklearn import linear_model   
model = linear_model.LinearRegression()
model.fit(train_data_padding, list_label_train)
predict_LinearRegression = model.predict(test_data_padding)


In [29]:

def predict_label_to_int(predict_label_list,threshold):

    predict_label_int = []
    for i in predict_label_list:
        value= i
        if value >threshold:label_ = int(1)
        else:label_ = int(0)
        predict_label_int.append(label_)

    return predict_label_int

In [30]:
from sklearn import linear_model   
model = linear_model.LinearRegression()
model.fit(train_data_padding, list_label_train)
predict_LinearRegression = predict_label_to_int(
    predict_label_list=model.predict(test_data_padding),
    threshold= 0.01)
measure(
    predict_LinearRegression,list_label_test)


0.866 0.8469


In [31]:
for i in [0.01,0.02,0.03,0.04,0.05]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8583 0.5553
0.8583 0.5553
0.8583 0.5553
0.8583 0.5553
0.8583 0.5553


In [32]:
for i in [0.1,0.2,0.3,0.4,0.5]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8583 0.5553
0.8583 0.5553
0.8583 0.5553
0.8583 0.5553
0.8583 0.5553


In [33]:
from sklearn import linear_model   
model = linear_model.LinearRegression()
model.fit(train_data_padding, list_label_train)
result = model.predict(test_data_padding)

In [34]:
for i in [0.1,0.2,0.3,0.4,0.5]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.869 0.8446
0.8744 0.8408
0.8837 0.8358
0.8913 0.8216
0.8948 0.8017


In [35]:
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.869 0.8446
0.8744 0.8408
0.8837 0.8358
0.8913 0.8216
0.8948 0.8017
0.8899 0.7717
0.8626 0.7322


In [36]:
#LOAD 
def prepare_label(dict_data)-> list:
    """
        dict_data : { e_id: {
                            'log_features':[] ,
                            'info_features':[] ,
                            'label' : 1 or 0
                            } 
                    }

    Returns:
        list_assemble_data : [[ *log_features , *info_features ],......]
        list_e_id : [ e_id_1 ,e_id_2,...... ]
        list_label : [ 0, 1, ..........]
    """    
    list_assemble_data = []
    list_e_id = []
    list_label = []
    for e_id,dict_ in dict_data.items():

        log_data  = dict_['log_features']
        info_data = dict_['info_features']
        label     = dict_['label']

        list_assemble_data.append([ *log_data,*info_data ])
        list_e_id.append(int(e_id))
        list_label.append(int(label))

    return list_assemble_data , list_e_id , list_label
def assemble_predictedal_and_predict_label(
    list_label,
    list_e_id,
    predict_label)-> dict:
    """
    Returns:
        dict: { e_id:[ predictedal_label , predict_label ]}
    """    
    dict_ori_and_predict_label = {}
    for row in range(len(list_label)):
        e_id = list_e_id[row]
        ori_label = list_label[row]
        dict_ori_and_predict_label[e_id] = [ori_label,predict_label[row]]

    return dict_ori_and_predict_label

def predict_label_to_int(predict_label_list,threshold):

    predict_label_int = []
    for i in predict_label_list:
        value= i
        if value >threshold:label_ = int(1)
        else:label_ = int(0)
        predict_label_int.append(label_)

    return predict_label_int
def measure(predict_label_int,list_label_test):
    f1             = f1_score(predict_label_int,list_label_test)
   # accuracy = accuracy_score(predict_label_int,list_label_test)
    AUC =       roc_auc_score(predict_label_int,list_label_test)
    # precision = precision_score(predict_label_int,test_label.tolist())
    # recall = recall_score(predict_label_int,test_label.tolist())

    print(round(f1,4),round(AUC,4))

import json
dict_data_train = json.load(open('after_processed_data_file\\dict_data_train_for_analy_fix_birth.json','r'))
dict_data_test = json.load(open('after_processed_data_file\\dict_data_test_for_analy_fix_birth.json','r'))

list_data_train,list_e_id_train,list_label_train = prepare_label(dict_data_train)
list_data_test,list_e_id_test,list_label_test = prepare_label(dict_data_test)

import numpy as np
train_data =np.array( list_data_train)
test_data  =np.array( list_data_test)
from sklearn.impute import SimpleImputer
Padding = SimpleImputer(missing_values=np.nan, strategy='mean')
train_data_padding = Padding.fit_transform(train_data)
test_data_padding  = Padding.fit_transform(test_data)


In [37]:
from sklearn import linear_model   
#linear_model.BayesianRidge()

model = linear_model.LinearRegression()
model.fit(train_data_padding, list_label_train)
result = model.predict(test_data_padding)

In [38]:
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8696 0.8436
0.8743 0.8393
0.8809 0.8313
0.8884 0.8216
0.8922 0.8005
0.8873 0.7661
0.8685 0.7322


In [40]:

from sklearn import linear_model   
#linear_model.BayesianRidge()

model = linear_model.LinearRegression(
    normalize=True
    ,n_jobs=-1)
model.fit(train_data_padding, list_label_train)
result = model.predict(test_data_padding)

In [41]:
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8696 0.8436
0.8743 0.8393
0.8809 0.8313
0.8884 0.8216
0.8922 0.8005
0.8873 0.7661
0.8685 0.7322


In [42]:
for i in [0.01,0.05,0.075,0.1,0.45,0.5,0.55,0.6 ]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8669 0.8482
0.8682 0.8466
0.8688 0.8434
0.8696 0.8436
0.891 0.814
0.8922 0.8005
0.8906 0.7817
0.8873 0.7661


In [43]:
for i in [0.001, 0.49,0.5,0.51 ]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8668 0.8509
0.8921 0.8033
0.8922 0.8005
0.892 0.7963


In [44]:
for i in [0.001, 0.495,0.5,0.505 ]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8668 0.8509
0.8921 0.8019
0.8922 0.8005
0.8919 0.7977


In [45]:
for i in [0.001, 0.499,0.5,0.501 ]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8668 0.8509
0.8922 0.801
0.8922 0.8005
0.8922 0.8001


In [46]:

from sklearn import linear_model   
bay = linear_model.BayesianRidge()
bay.fit(train_data_padding, list_label_train)
result_bay = bay.predict(test_data_padding)

In [47]:
for i in [0.001, 0.499,0.5,0.501 ]:
    predict_LinearRegression = predict_label_to_int(
        result_bay,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8668 0.851
0.8921 0.8007
0.8921 0.8004
0.8922 0.8003


In [48]:
model.rank_

33

In [49]:
model.coef_

array([-6.95626262e-07,  3.78412327e-13, -6.16354821e-02,  6.93101649e-03,
       -1.93263421e-03,  1.87253767e-07, -1.95668043e-02,  3.10247895e-04,
       -1.12285668e-05, -5.16143230e-02, -7.82968360e-02, -1.27053239e-02,
        4.99977613e-02, -3.29189321e-05, -1.33033256e-02,  3.02063016e-02,
        3.64242152e-02, -4.54485230e-02, -1.55959567e-04,  5.19382963e-02,
        1.22203076e-02, -3.10052242e-02, -5.54236503e-02, -1.30756218e-03,
       -1.77658712e-02, -2.13416308e-03, -2.22020463e-03,  7.21602919e-04,
       -8.97060204e-13, -1.61490220e-09,  1.64478010e-05, -8.33870469e-04,
        5.91919769e-03,  6.75841022e-03])

In [50]:
model.intercept_

0.1114952978474485

In [51]:
model.singular_

array([2.50302241e+00, 1.65889304e+00, 1.51659859e+00, 1.39438799e+00,
       1.37533018e+00, 1.33063379e+00, 1.29145872e+00, 1.19920296e+00,
       1.15970341e+00, 1.12813742e+00, 1.07046614e+00, 1.05340525e+00,
       1.00323629e+00, 9.83738141e-01, 9.25789122e-01, 9.11970090e-01,
       8.87049851e-01, 7.32331572e-01, 6.97012350e-01, 6.62516700e-01,
       6.51455891e-01, 5.97915652e-01, 5.86476028e-01, 4.70581954e-01,
       4.52536299e-01, 4.38906699e-01, 3.89548511e-01, 3.37182652e-01,
       2.99156334e-01, 2.89475277e-01, 5.13471760e-02, 1.41817154e-02,
       1.04010273e-02, 1.95265170e-16])